In [1]:
import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.facecolor'] = 'white'

if not os.path.abspath('../util') in sys.path:
    sys.path.append(os.path.abspath('../util'))
import util as yu

ens='cA211.53.24'
lat=yu.LatticeEnsemble(ens)

In [2]:
# load data

basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/NJNpi_N0pi+/proj/'
with h5py.File(basePath+'N_2pt.h5') as f:
    cfgList=list(yu.deepKey(f,3).keys())

basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/NJN/proj/'
with h5py.File(basePath+'dt14_protonup.h5') as f:
    cfgList=list(set(cfgList)&set(yu.deepKey(f,3).keys()))
with h5py.File(basePath+'dt14_protondn.h5') as f:
    cfgList=list(set(cfgList)&set(yu.deepKey(f,3).keys()))

print(len(cfgList))

basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/NJNpi_N0pi+/proj/'

filesDic={}
files2ptKey=['NN']
filesDic['NN']=['N_2pt']
filesDic['NNpi']=['T']
filesDic['NpiNpi']=['B_2pt','W_2pt','Z_2pt','M_correct_2pt']
files3ptKey=[]
filesDic['NJNpi']=['B','W','Z']

f={}
for key in filesDic.keys():
    f[key]={}
    for file in filesDic[key]:
        f[key][file] = h5py.File(basePath + file + '.h5','r')

data={}
for key in files2ptKey:
    data[key]={}
    for file in filesDic[key]:
        data[key][file]={}
        tF=f[key][file]
        for projAStr in tF.keys():
            projA=eval(projAStr)
            data[key][file][projA]={}
            for projBStr in tF[projAStr].keys():
                projB=eval(projBStr)
                data[key][file][projA][projB]={}
                for proj2 in tF[projAStr][projBStr].keys():
                    data[key][file][projA][projB][proj2]={}
                    for cont in tF[projAStr][projBStr][proj2][cfgList[0]].keys():
                        data[key][file][projA][projB][proj2][cont]=np.array([
                            f[key][file][projAStr][projBStr][proj2][cfg][cont][()] for cfg in cfgList
                            ])

for key in files3ptKey:
    data[key]={}
    for file in filesDic[key]:
        data[key][file]={}
        tF=f[key][file]
        for projAStr in tF.keys():
            projA=eval(projAStr)
            data[key][file][projA]={}
            for projBStr in tF[projAStr].keys():
                projB=eval(projBStr)
                data[key][file][projA][projB]={}
                for proj2 in tF[projAStr][projBStr].keys():
                    data[key][file][projA][projB][proj2]={}
                    for cont in tF[projAStr][projBStr][proj2][cfgList[0]].keys():
                        data[key][file][projA][projB][proj2][cont]=np.array([
                            f[key][file][projAStr][projBStr][proj2][cfg][cont][()] for cfg in cfgList
                            ])

for key in filesDic.keys():
    for file in filesDic[key]:
        f[key][file].close()


basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/NJN/proj/'
filesDic={}
files3ptKey=['NJN']
filesDic['NJN']=['dt10_protonup','dt10_protondn','dt12_protonup','dt12_protondn','dt14_protonup','dt14_protondn']

f={}
for key in filesDic.keys():
    f[key]={}
    for file in filesDic[key]:
        f[key][file] = h5py.File(basePath + file + '.h5','r')

for key in files3ptKey:
    data[key]={}
    for file in filesDic[key]:
        data[key][file]={}
        tF=f[key][file]
        for projAStr in tF.keys():
            projA=eval(projAStr)
            data[key][file][projA]={}
            for projBStr in tF[projAStr].keys():
                projB=eval(projBStr)
                data[key][file][projA][projB]={}
                for proj2 in tF[projAStr][projBStr].keys():
                    data[key][file][projA][projB][proj2]={}
                    for cont in tF[projAStr][projBStr][proj2][cfgList[0]].keys():
                        data[key][file][projA][projB][proj2][cont]=np.array([
                            f[key][file][projAStr][projBStr][proj2][cfg][cont][()] for cfg in cfgList
                            ])

for key in filesDic.keys():
    for file in filesDic[key]:
        f[key][file].close()


308


In [3]:
projA=((0,0,0),'a');projB=((0,0,0),'a');proj2='P0'
data['NN']['N_2pt'][projA][projB][proj2]['NP'].shape

(308, 16)

In [ ]:
def getDat():
    projA=((0,0,0),'a');projB=((0,0,0),'a');proj2='P0'
    return data['NN']['N_2pt'][projA][projB][proj2]['NP']

dat=getDat()
def tFunc(dat):
    t=np.mean(dat,axis=0)
    t=np.real(t)
    t=np.log(t/np.roll(t,-1))
    return t
(mean,cov,err)=yu.jackknife(dat,tFunc)

(Nrow, Ncol) = (1, 1)
fig, axs = plt.subplots(Nrow, Ncol, figsize=(5*Ncol, 4*Nrow), squeeze=False)
irow=0;icol=0
tmin=1;tmax=14+1
tMean=mean[tmin:tmax];tErr=err[tmin:tmax]
axs[irow,icol].errorbar(np.arange(tmin,tmax),tMean,tErr,fmt='s',elinewidth=2,capsize=6,mfc='white')


fitMin=9;fitMax=14+1
fitCov=cov[fitMin:fitMax,fitMin:fitMax]
def tFunc(dat):
    t=np.mean(dat,axis=0)
    t=np.real(t)
    t=np.log(t/np.roll(t,-1))

    xList=np.arange(fitMin,fitMax)
    fitData=t[fitMin:fitMax]
    fitFunc=lambda tf,mN:mN
    paraInit=[0.55]
    fitMean=curve_fit(fitFunc,xList,fitData,paraInit,sigma=fitCov)[0]

    chi2Pre=fitFunc(xList,*fitMean)-fitData
    chi2=chi2Pre.dot(np.linalg.inv(fitCov)).dot(chi2Pre.T)
    dof=len(fitData)-len(paraInit)
    chi2Dof=chi2/dof

    return np.array([fitMean]+[np.array([chi2,dof,chi2Dof])],dtype=object)
(mean,cov,err)=yu.jackknife(dat,tFunc,2)
(mean,err)

In [ ]:
def getDat3pt(Gc,iso,tf):
    projA=((0,0,0),'a');projB=((0,0,0),'a');proj='P0'
    i_Gc={'id':0,'gx':1,'gy':2,'gz':3,'gt':4,'g5':5,'g5gx':6,'g5gy':7,'g5gz':8,'g5gt':9}[Gc]
    sgn_iso={'+':+1,'-':-1}[iso]
    return data['NJN']['dt'+str(tf)+'_protonup'][projA][projB][proj]['MprotonUp'][:,1:tf,i_Gc]+sgn_iso*data['NJN']['dt'+str(tf)+'_protondn'][projA][projB][proj]['MprotonDn'][:,1:tf,i_Gc]

tfList=[10,12,14]
Ntf=len(tfList)
def getDat():
    Gc='id';iso='+'
    return np.array([np.array([getDat3pt(Gc,iso,tf)[cfg] for tf in tfList],dtype=object) for cfg in range(len(cfgList))])

preFactor=1
dat=getDat()

def tFunc(dat):
    t=np.mean(dat,axis=0)
    t=np.array([np.real(preFactor*t[ind_tf]) for ind_tf in range(Ntf)],dtype=object)
    return t

(mean,cov,err)=yu.jackknife(dat,tFunc,2)

(Nrow, Ncol) = (1, 1)
fig, axs = plt.subplots(Nrow, Ncol, figsize=(5*Ncol, 4*Nrow), squeeze=False)
irow=0;icol=0
for i in range(len(tfList)):
    tf=tfList[i]
    tMean=mean[i];tErr=err[i]
    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,fmt='s',elinewidth=2,capsize=6,mfc='white')

threept=lambda t2,t1,E0,E1,A00,A01,A11: A00*np.exp(-E0*t2)+A01*np.exp(-E0*t2)*np.exp(-(E1-E0)*t1)\
    +A01*np.exp(-E1*t2)*np.exp(-(E0-E1)*t1)+A11*np.exp(-E1*t2)

tcmin=2
fitCov=np.block([[cov[i,j][tcmin-1:tfList[i]-tcmin,tcmin-1:tfList[j]-tcmin] for j in range(Ntf)] for i in range(Ntf)])
def tGet(t,ind_tf):
    t_f=tfList[ind_tf]
    return np.real(t[ind_tf][(tcmin-1):t_f-tcmin])
def tFunc(dat):
    t=np.mean(dat,axis=0)
    t=np.array([tGet(t,ind_tf) for ind_tf in range(len(tfList))],dtype=object)

    fitData=np.concatenate(t)
    tftcList=[[[tf,tc] for tc in range(tcmin,tf-tcmin+1)] for tf in tfList ]
    tftcList=np.concatenate(tftcList).T
    fitFunc=lambda tftc,E0,E1,A00,A01,A11:threept(tftc[0],tftc[1],E0,E1,A00,A01,A11)
    paraInit=[0.5,1,1e-5,1e-5,1e-5]
    fitMean=curve_fit(fitFunc,tftcList,fitData,paraInit,sigma=fitCov)[0]

    chi2Pre=fitFunc(tftcList,*fitMean)-fitData
    chi2=chi2Pre.dot(np.linalg.inv(fitCov)).dot(chi2Pre.T)
    dof=len(fitData)-len(paraInit)
    chi2Dof=chi2/dof

    return np.array([fitMean]+[np.array([chi2,dof,chi2Dof])],dtype=object)

(mean,cov,err)=yu.jackknife(dat,tFunc,2)
print(mean)
print(err)

for tf in tfList:
    tFunc=lambda tc:threept(tf,tc,*mean[0])
    tcList=np.arange(tcmin,tf-tcmin+1)
    axs[irow,icol].plot(tcList-tf//2,tFunc(tcList),color='r')

    tFunc=lambda tc:threept(tf,tc,mean[0][0],mean[0][1],mean[0][2],0*mean[0][3],-1*mean[0][4])
    tcList=np.arange(tcmin,tf-tcmin+1)
    axs[irow,icol].plot(tcList-tf//2,tFunc(tcList),color='g')

    tFunc=lambda tc:threept(tf,tc,mean[0][0],mean[0][1],mean[0][2],0*mean[0][3],0*mean[0][4])
    tcList=np.arange(tcmin,tf-tcmin+1)
    axs[irow,icol].plot(tcList-tf//2,tFunc(tcList),color='b')

In [ ]:
def getDat3pt(Gc,iso,tf):
    projA=((0,0,0),'a');projB=((0,0,0),'a');proj='P0'
    i_Gc={'id':0,'gx':1,'gy':2,'gz':3,'gt':4,'g5':5,'g5gx':6,'g5gy':7,'g5gz':8,'g5gt':9}[Gc]
    sgn_iso={'+':+1,'-':-1}[iso]
    return data['NJN']['dt'+str(tf)+'_protonup'][projA][projB][proj]['MprotonUp'][:,1:tf,i_Gc]+sgn_iso*data['NJN']['dt'+str(tf)+'_protondn'][projA][projB][proj]['MprotonDn'][:,1:tf,i_Gc]

tfList=[10,12,14]
Ntf=len(tfList)
def getDat():
    Gc='id';iso='-'
    return np.array([np.array([getDat3pt(Gc,iso,tf)[cfg] for tf in tfList],dtype=object) for cfg in range(len(cfgList))])

preFactor=1
dat=getDat()

def tFunc(dat):
    t=np.mean(dat,axis=0)
    t=np.array([np.real(preFactor*t[ind_tf]) for ind_tf in range(Ntf)],dtype=object)
    return t

(mean,cov,err)=yu.jackknife(dat,tFunc,2)

(Nrow, Ncol) = (1, 2)
fig, axs = plt.subplots(Nrow, Ncol, figsize=(5*Ncol, 4*Nrow), squeeze=False)
irow=0;icol=0
for i in range(len(tfList)):
    tf=tfList[i]
    tMean=mean[i];tErr=err[i]
    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,fmt='s',elinewidth=2,capsize=6,mfc='white')
    
threept=lambda t2,t1,E0,E1,A00,A01,A11: A00*np.exp(-E0*t2)+A01*np.exp(-E0*t2)*np.exp(-(E1-E0)*t1)\
    +A01*np.exp(-E1*t2)*np.exp(-(E0-E1)*t1)+A11*np.exp(-E1*t2)

tcmin=2
fitCov=np.block([[cov[i,j][tcmin-1:tfList[i]-tcmin,tcmin-1:tfList[j]-tcmin] for j in range(Ntf)] for i in range(Ntf)])
def tGet(t,ind_tf):
    t_f=tfList[ind_tf]
    return np.real(t[ind_tf][(tcmin-1):t_f-tcmin])
def tFunc(dat):
    t=np.mean(dat,axis=0)
    t=np.array([tGet(t,ind_tf) for ind_tf in range(len(tfList))],dtype=object)

    fitData=np.concatenate(t)
    tftcList=[[[tf,tc] for tc in range(tcmin,tf-tcmin+1)] for tf in tfList ]
    tftcList=np.concatenate(tftcList).T
    fitFunc=lambda tftc,E0,E1,A00,A01,A11:threept(tftc[0],tftc[1],E0,E1,A00,A01,A11)
    paraInit=[0.5,1,1e-5,1e-5,1e-5]
    fitMean=curve_fit(fitFunc,tftcList,fitData,paraInit,sigma=fitCov)[0]

    chi2Pre=fitFunc(tftcList,*fitMean)-fitData
    chi2=chi2Pre.dot(np.linalg.inv(fitCov)).dot(chi2Pre.T)
    dof=len(fitData)-len(paraInit)
    chi2Dof=chi2/dof

    return np.array([fitMean]+[np.array([chi2,dof,chi2Dof])],dtype=object)

(mean,cov,err)=yu.jackknife(dat,tFunc,2)
print(mean)
print(err)

for tf in tfList:
    tFunc=lambda tc:threept(tf,tc,*mean[0])
    tcList=np.arange(tcmin,tf-tcmin+1)
    axs[irow,icol].plot(tcList-tf//2,tFunc(tcList),color='r')

    tFunc=lambda tc:threept(tf,tc,mean[0][0],mean[0][1],mean[0][2],0*mean[0][3],-1*mean[0][4])
    tcList=np.arange(tcmin,tf-tcmin+1)
    axs[irow,icol].plot(tcList-tf//2,tFunc(tcList),color='g')

    tFunc=lambda tc:threept(tf,tc,mean[0][0],mean[0][1],mean[0][2],0*mean[0][3],0*mean[0][4])
    tcList=np.arange(tcmin,tf-tcmin+1)
    axs[irow,icol].plot(tcList-tf//2,tFunc(tcList),color='b')

In [6]:
lat.getaEpi(1)+lat.getaEN(1)

0.9287144475540623